In [ ]:
from dotenv import load_dotenv
load_dotenv()
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.documents import Document
from langchain_core.output_parsers import StrOutputParser
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_core.tools import tool
from langchain.agents import create_agent
from langgraph.checkpoint.memory import InMemorySaver


In [35]:
# llm=ChatGoogleGenerativeAI(
#     model="gemini-2.5-flash",
#     temperature=0.2
# )

In [36]:
documents = [
    Document(
        page_content=(
            "Retrieval Augmented Generation, or RAG, is a technique where a large "
            "language model retrieves relevant information from an external "
            "knowledge base and uses it as context to generate accurate answers."
        )
    ),
    Document(
        page_content=(
            "LangChain is a framework that helps developers build applications "
            "powered by large language models by chaining prompts, models, "
            "retrievers, tools, and memory together."
        )
    ),
    Document(
        page_content=(
            "Vector databases store numerical embeddings of text and allow "
            "semantic similarity search instead of exact keyword matching."
        )
    )
]


splitter = RecursiveCharacterTextSplitter(
    chunk_size=200,
    chunk_overlap=20
)
chunks = splitter.split_documents(documents)

In [37]:
embeddings = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-MiniLM-L6-v2"
)

In [38]:
vectorstore = FAISS.from_documents(chunks, embeddings)
retriever = vectorstore.as_retriever(search_kwargs={"k": 3})

In [39]:
@tool
def knowledge_search(query: str) -> str:
    """
    Search the knowledge base for factual or technical information.
    """
    docs = retriever.invoke(query)

    if not docs:
        return "I don't know"

    return "\n".join(d.page_content for d in docs)

@tool
def summarize_text(text: str) -> str:
    """
    Summarize the given text in simple words.
    """
    llm = ChatGoogleGenerativeAI(
        model="gemini-1.5-flash",
        temperature=0.3
    )
    return llm.invoke(f"Summarize this clearly:\n{text}").content

@tool
def calculator(expression: str) -> str:
    """
    Evaluate a simple mathematical expression.
    Example: '12 * 8'
    """
    try:
        return str(eval(expression))
    except Exception:
        return "Invalid mathematical expression"



In [40]:
llm=ChatGoogleGenerativeAI(
    model="gemini-2.5-flash",
    temperature=0.2
)

In [41]:
# prompt = ChatPromptTemplate.from_template(
#     """
#     Use ONLY the context below to answer the question.
#     If the answer is not in the context, say "I don't know".

#     Context:
#     {context}

#     Question:
#     {question}
#     """
# )
# output_parser = StrOutputParser()


In [42]:
agent_prompt = ChatPromptTemplate.from_messages([
    (
        "system",
        """
        You are a Research AI assistant.

        Rules:
        - Use knowledge_search for factual or technical questions.
        - Use summarize_text to simplify long answers.
        - Use calculator for math calculations.
        - If information is not found, say "I don't know".
        - Do NOT hallucinate.
        """
    ),
    ("human", "{input}")
])



In [43]:
# chain = prompt | llm | output_parser
# def ask(question: str):
#     docs = retriever.invoke(question)
#     context = "\n".join(d.page_content for d in docs)
#     return chain.invoke({"context": context, "question": question})


# print(ask("Explain retrieval augmented generation in simple terms"))
# print(ask("How does LangChain help in building LLM applications?"))
# print(ask("Tell me about vector databases"))
# print(ask("Who is Tejas?"))


In [44]:
tools = [knowledge_search, summarize_text, calculator]

llm_with_tools = llm.bind_tools(tools)

chain = agent_prompt | llm_with_tools



In [45]:
def research_agent(question: str):
    response = chain.invoke({"input": question})

    if response.tool_calls:
        for tool_call in response.tool_calls:
            name = tool_call["name"]
            args = tool_call["args"]

            if name == "knowledge_search":
                result = knowledge_search.invoke(args["query"])
                return result

            if name == "summarize_text":
                return summarize_text.invoke(args["text"])

            if name == "calculator":
                return calculator.invoke(args["expression"])

    return response.content




In [46]:
print(research_agent("Explain retrieval augmented generation in simple terms"))
print(research_agent("Summarize what LangChain is"))
print(research_agent("What is 12 * 8?"))
print(research_agent("Who is Tejas?"))



Retrieval Augmented Generation, or RAG, is a technique where a large language model retrieves relevant information from an external knowledge base and uses it as context to generate accurate answers.
Vector databases store numerical embeddings of text and allow semantic similarity search instead of exact keyword matching.
LangChain is a framework that helps developers build applications powered by large language models by chaining prompts, models, retrievers, tools, and memory together.
LangChain is a framework that helps developers build applications powered by large language models by chaining prompts, models, retrievers, tools, and memory together.
Retrieval Augmented Generation, or RAG, is a technique where a large language model retrieves relevant information from an external knowledge base and uses it as context to generate accurate answers.
Vector databases store numerical embeddings of text and allow semantic similarity search instead of exact keyword matching.
96
LangChain is 